# Отток клиентов банка

**Описание проекта**

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

План работы:

1. Загрузите и подготовьте данные.
2. Исследуйте баланс классов, обучите модель без учёта дисбаланса.
3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую.
4. Проведите финальное тестирование.

**Описание данных**

Признаки

1. `RowNumber` — индекс строки в данных
2. `CustomerId` — уникальный идентификатор клиента
3. `Surname` — фамилия
4. `CreditScore` — кредитный рейтинг
5. `Geography` — страна проживания
6. `Gender` — пол
7. `Age` — возраст
8. `Tenure` — сколько лет человек является клиентом банка
9. `Balance` — баланс на счёте
10. `NumOfProducts` — количество продуктов банка, используемых клиентом
11. `HasCrCard` — наличие кредитной карты
12. `IsActiveMember` — активность клиента
13. `EstimatedSalary` — предполагаемая зарплата

Целевой признак

1. `Exited` — факт ухода клиента

In [2]:
import pandas as pd
import optuna
import plotly.express as px

from caseconverter import snakecase
from ydata_profiling import ProfileReport
from IPython.display import display
from tqdm import tqdm

from fast_ml import eda
from fast_ml.model_development import train_valid_test_split

from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, roc_curve, roc_auc_score
)

In [3]:
FIG_WIDTH = 8
FIG_HEIGHT = 5
RANDOM_SEED = 42

In [4]:
try:
    raw_users = pd.read_csv('Churn.csv')
except:
    raw_users = pd.read_csv('/datasets/Churn.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [5]:
display(eda.df_info(raw_users))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
RowNumber,int64,Numerical,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
CustomerId,int64,Numerical,10000,"[15634602, 15647311, 15619304, 15701354, 15737...",0,0.0
Surname,object,Categorical,2932,"[Hargrave, Hill, Onio, Boni, Mitchell, Chu, Ba...",0,0.0
CreditScore,int64,Numerical,460,"[619, 608, 502, 699, 850, 645, 822, 376, 501, ...",0,0.0
Geography,object,Categorical,3,"[France, Spain, Germany]",0,0.0
Gender,object,Categorical,2,"[Female, Male]",0,0.0
Age,int64,Numerical,70,"[42, 41, 39, 43, 44, 50, 29, 27, 31, 24]",0,0.0
Tenure,float64,Numerical,11,"[2.0, 1.0, 8.0, 7.0, 4.0, 6.0, 3.0, 10.0, 5.0,...",909,9.09
Balance,float64,Numerical,6382,"[0.0, 83807.86, 159660.8, 125510.82, 113755.78...",0,0.0
NumOfProducts,int64,Numerical,4,"[1, 3, 2, 4]",0,0.0


Числовые распределения:

In [6]:
display(round(raw_users.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5000.50,2886.90,1.00,2500.75,5000.50,7500.25,10000.00
CustomerId,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
CreditScore,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
Age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
Tenure,9091.0,5.00,2.89,0.00,2.00,5.00,7.00,10.00
Balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
NumOfProducts,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
HasCrCard,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
IsActiveMember,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
EstimatedSalary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48


И детальный отчет:

In [7]:
ProfileReport(raw_users).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Ключевые наблюдения из предварительного анализа набора данных:

1. **Демографические данные клиентов**: Набор данных содержит информацию о 10 000 клиентах с различными характеристиками. Средний возраст клиентов составляет 39 лет, с разбросом от 18 до 92 лет. В наборе данных представлены клиенты из трех стран: Франции, Испании и Германии. Кроме того, присутствует почти равномерное распределение мужчин и женщин.

2. **Финансовые показатели**: Набор данных включает финансовые метрики, такие как кредитные баллы, балансы счетов, количество продуктов и зарплаты. Средний кредитный балл составляет 651, что указывает на относительно хорошую кредитоспособность. Средний баланс счета составляет 76К долларов, с широким диапазоном значений. Клиенты имеют в среднем 1,53 продукта в банке, а зарплаты варьируются от 11 до 200К долларов.

3. **Отток и вовлеченность**: Около 20% клиентов в наборе данных покинули или прекратили использовать услуги банка. Приблизительно 52% клиентов являются активными участниками, что указывает на умеренный уровень вовлеченности. Средняя продолжительность работы с клиентом составляет 5 лет, хотя в этой колонке есть пропущенных значений для некоторых записей.

4. **Целевой признак**: У нас сть дисбаланс классов в целевом признаке `exited` - около 8К записей оставшихся пользователей и только 2К записей пользователей, которые перестали быть пользователями нашего банка. Это надо будет выровнять, чтобы избежать model bias.

# Подготовка данных для ML

Выполним эти преобразования на полном датасете:

1. Уберем колонки, которые не нужны для моделей: `RowNumber`, `CustomerId`, `Surname`.
2. Переведем колонку `gender` в бинарную: `1` - для мужчин и `0` - для женщин.
3. Названия колонок приведем `snake_case` регистру.
4. Проведем `upsampling`.

И остальные после разделения на выборки, чтобы избежать data leakage:

1. Заполним пропуски в `tenure`.
2. Проведем стандартизацию численных признаков.
3. Проведем кодирование категориальных признаков.

In [34]:
df_no_sampling = (
    raw_users
    .copy()
    .drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
    .replace({'Male': 1, 'Female': 0})
    .rename(columns=lambda column: snakecase(column))
    .assign(geography=lambda df: df.geography.str.lower())
)

display(df_no_sampling.head())


,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,france,0,42,2.0,0.00,1,1,1,101348.88,1
1,608,spain,0,41,1.0,83807.86,1,0,1,112542.58,0
2,502,france,0,42,8.0,159660.80,3,1,0,113931.57,1
3,699,france,0,39,1.0,0.00,2,0,0,93826.63,0
4,850,spain,0,43,2.0,125510.82,1,1,1,79084.10,0


Выглядит адекватно. Сделаем `upsampling`.

In [9]:
# Perform the split between majority and minority, upsample minority, and concatenate results
df_upsampled = (
    pd.concat([
        df_no_sampling[df_no_sampling.exited==0],
        resample(
            df_no_sampling[df_no_sampling.exited==1],
            replace=True,
            n_samples=len(df_no_sampling[df_no_sampling.exited==0]),
            random_state=RANDOM_SEED
        )
    ])
)

Проверим, что получилось.

In [10]:
def print_class_distribution(df, column, label):
    print(
        f"{'-' * 40}\n{label} distribution, %:\n{df[column].value_counts(normalize=True) * 100}\n{'-' * 40}"
    )

print_class_distribution(df_no_sampling, 'exited', 'Exited without resampling')
print_class_distribution(df_upsampled, 'exited', 'Exited with upsampling')

----------------------------------------
Exited without resampling distribution, %:
0    79.63
1    20.37
Name: exited, dtype: float64
----------------------------------------
----------------------------------------
Exited with upsampling distribution, %:
0    50.0
1    50.0
Name: exited, dtype: float64
----------------------------------------


В `df_upsampled` классы теперь сбалансированы.

Разделим датасеты на выборки и выполним остальные преобразовния.

In [36]:
# Define variables
data_splits = {}
split_keys = ['ftr_train', 'tgt_train', 'ftr_valid', 'tgt_valid', 'ftr_test', 'tgt_test']
num_cols = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']
cat_cols = ['geography']
bin_cols = ['has_cr_card', 'is_active_member', 'gender']

# Define the preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols),
])


for dataset, label in zip([df_no_sampling, df_upsampled], ['no_sampling', 'upsampled']):
    splits = train_valid_test_split(
        dataset, 'exited', train_size=0.7, valid_size=0.15, test_size=0.15, random_state=RANDOM_SEED
    )
    
    # Use the split keys and the splits to create a dictionary for each dataset
    data_splits[label] = dict(zip(split_keys, splits))
    
    # Fit the preprocessor on the training data only
    preprocessor.fit(data_splits[label]['ftr_train'])

    # Transform and reformat each subset of the data
    for subset in split_keys:
        # Only transform feature subsets
        if 'ftr' in subset:
            data_transformed = preprocessor.transform(data_splits[label][subset])
            columns_transformed = num_cols + list(
                preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(cat_cols)
            )

            # Create a DataFrame from the transformed data
            df_transformed = pd.DataFrame(data_transformed, columns=columns_transformed)

            # Add the binary columns back to the DataFrame
            df_transformed = pd.concat([df_transformed, data_splits[label][subset][bin_cols].reset_index(drop=True)], axis=1)

            # Replace the original data with the transformed data in the data_splits dictionary
            data_splits[label][subset] = df_transformed

Проверим, что получилось на одном примере.

In [44]:
data_splits['no_sampling']['ftr_valid'].head()

,credit_score,age,tenure,balance,num_of_products,estimated_salary,geography_france,geography_germany,geography_spain,has_cr_card,is_active_member,gender
0,-0.012043,-0.371195,1.135650,0.538578,-0.903352,0.465751,0.0,1.0,0.0,1,0,1
1,-0.012043,-0.466380,-0.208087,0.242578,-0.903352,0.401147,1.0,0.0,0.0,1,1,1
2,-1.643626,-0.085639,0.799716,1.576231,-0.903352,-1.053519,0.0,1.0,0.0,1,0,0
3,0.829729,1.437322,-1.551825,0.925391,-0.903352,-1.516265,0.0,0.0,1.0,0,1,1
4,0.091879,-0.371195,0.127847,0.832067,-0.903352,0.725006,0.0,0.0,1.0,1,0,0


В этом скрипте мы успешно обработали два набора данных, `df_no_sampling` и `df_upsampled`. На первом этапе были определены переменные и настроен предпроцессор предварительной обработки для числовых и категориальных столбцов, бинарные столбцы остались без изменений.

Основной цикл скрипта выполняет три основные задачи для каждого набора данных:

1. Разделение данных: Используется функция `train_valid_test_split`, чтобы разделить каждый набор данных на обучающие, валидационные и тестовые подмножества.

2. Подгонка препроцессора: Препроцессор обучается на обучающих данных. Это включает в себя изучение параметров для `imputation` наиболее часто встречающихся значений и стандартного масштабирования.

3. Трансформация данных: Используя подготовленный препроцессор, трансформируются признаки для каждого набора и типа колонок.

После выполнения этого скрипта у нас есть все подмножества данных, готовые для дальнейшего обучения модели и оценки.

# ML модели

Перейдем к обучению моделей.